# Import librairies

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
# !pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# scrape the Wikipedia page

In [5]:
# import required modules 
from bs4 import BeautifulSoup 

# send the GET request
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text


In [6]:
#print(page)

In [7]:
# scrape webpage 
soup = BeautifulSoup(page, 'html.parser') 
  

In [8]:
my_table = soup.find('table')
# (my_table)


In [9]:
PostalCodes = []
Boroughs = []
Neighborhoods = []

# for each row of the table, find all the table data
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')

In [10]:
# append the data 
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        PostalCodes.append(cells[0].text)
        Boroughs.append(cells[1].text)
        Neighborhoods.append(cells[2].text.rstrip('\n')) # avoid new lines in neighborhood cell

In [11]:
# create a new DataFrame 
df_toronto = pd.DataFrame({"PostalCode": PostalCodes,
                           "Borough": Boroughs,
                           "Neighborhood": Neighborhoods})

df_toronto = df_toronto.replace('\n',' ', regex=True)
df_toronto.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
df_toronto.Borough = df_toronto.Borough.str.strip()

df_toronto['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [13]:
# remove rows with borough that is Not assigned.

df_toronto1 = df_toronto[df_toronto.Borough != "Not assigned"].reset_index(drop=True)
df_toronto1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


# Group neighborhoods in the same postal code area

In [15]:
df_toronto_grouped = df_toronto1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough

In [16]:
# if Neighborhood="Not assigned", put the value the same as Borough
for index, row in df_toronto_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row["Borough"]
        
df_toronto_grouped.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


# Print the number of rows of your dataframe

In [18]:
df_toronto_grouped.shape

(103, 3)